# 1. Pose基础操作

In [ ]:
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
init()

In [ ]:
# 读取PDB文件，Local/Internet
from pyrosetta import *
from pyrosetta.toolbox import pose_from_rcsb
pose = pose_from_pdb("inputs/5tj3.pdb")
pose_internet = pose_from_rcsb("5TJ3")

pose.sequence() # get sequence
pose.annotated_sequence() #可以对sequence进行注释，但是注释结果我表示看不懂。。
pose.total_residue() # 等价于len(pose.sequence())
pose.pdb_info().chain(24) # 第24个residue所在的chain
pose.pdb_info().number(24) # 第24个residue的编号
pose.residue(24).name() # 第24个residue的氨基酸名称
pose.pdb_info().pdb2pose('A', 24) # PDB numbering to Pose numbering, which requires a chain ID and a residue number, 根据PDB的numbering规则找残基
pose.pdb_info().pose2pdb(24) # Pose numbering to PDB numbering，根据pose的numbering规则找残基

# 从一级序列生成线性结构
from pyrosetta import pose_from_sequence
seq_pose = pose_from_sequence('AAAAA')

res_24 = pose.residue(24) # 获取第24个残基的所有信息
atom_index = res_24.atom_index("CA") # 找到第24个残基中的CA原子的index
res_24.atom_is_backbone(atom_index) # 根据上述index判断该原子是否属于backbone


pdb_pose.dump_pdb('./data/output.pdb') # 从Pose输出pdb


# 获取各种结构信息
resid = pose.pdb_info().pdb2pose('A', 28)
print("phi:", pose.phi(resid))
print("psi:", pose.psi(resid))
print("chi1:", pose.chi(1, resid))


# 针对某个残基计算两个atom之间的length
res_28 = pose.residue(resid)
N28 = AtomID(res_28.atom_index("N"), resid)
CA28 = AtomID(res_28.atom_index("CA"), resid)
C28 = AtomID(res_28.atom_index("C"), resid)
print(pose.conformation().bond_length(N28, CA28))
print(pose.conformation().bond_length(CA28, C28))
# 也可以手动计算
N_xyz = res_28.xyz("N")
CA_xyz = res_28.xyz("CA")
C_xyz = res_28.xyz("C")
N_CA_vector = CA_xyz - N_xyz
CA_C_vector = CA_xyz - C_xyz
print(N_CA_vector.norm())
print(CA_C_vector.norm())
#用这个命令可以查询这函数的用法
conformation.bond_length? 
# 获得两个原子之间的angle
angle = pose.conformation().bond_angle(N28, CA28, C28)
pose.set_phi(residue_id, desired_angle)

# 展示如何发送pose到pymol中
from pyrosetta.rosetta.protocols.moves import PyMOLMover
pymover = PyMOLMover()
pymover.update_energy(True)
pymover.apply(pdb_pose)

# 标准PDB文件中包含多种信息，详见https://www.biostat.jhsph.edu/~iruczins/teaching/260.655/links/pdbformat.pdf
# 可以从中提取ATOM信息
from pyrosetta.toolbox import cleanATOM
cleanATOM("inputs/5tj3.pdb")

# 查看pdb中包含哪些chains
print(pose.pdb_info())
# 或者
print(pyrosetta.rosetta.core.pose.conf2pdb_chain(pose))
for k, v in pyrosetta.rosetta.core.pose.conf2pdb_chain(pose).items():
    print(k)
    print(v)
    print("\n")


# 比对两条sequence的差异
def compare_sequence(seq1, seq2):
    len1 = len(seq1)
    len2 = len(seq2)

    if len1>len2:
        seq_short = seq2
        seq_long = seq1
        print(f"short sequence: seq2, long sequence: seq1")
    else:
        seq_short = seq1
        seq_long = seq2
        print(f"short sequence: seq1, long sequence: seq2")
    for i in range(0, len(seq_short)):
        if seq_short[i]!=seq_long[i]: print(f"location-{i}: seq1-{seq_short[i]}, seq2-{seq_long[i]}")
    
    for i in range(len(seq_short), len(seq_long)):
        print(f"append on long sequence-{seq_long[i]}")

compare_sequence(seq1=pose.sequence(), seq2=pose_clean.sequence())

In [ ]:
# 关于antibody的一些基础操作

# 首先可以在init中采用`-input_ab_scheme` 来指定 `AHo_Scheme`
init('-input_ab_scheme AHo_Scheme')


# 查看antibody的各种信息
ab_info = antibody.AntibodyInfo(pose, antibody.AHO_Scheme, antibody.North)
print(ab_info)

for i in range(1, 7):
    print(i, ab_info.get_CDR_name(antibody.CDRNameEnum(i)))

for cdr in ['L1', 'l1', 'L2', 'l2', 'L3', 'H1', 'H2', 'H3']:
    print(cdr, str(ab_info.get_CDR_name_enum(cdr)))

print(str(antibody.h3))
print(int(antibody.h3))

enum_manager = antibody.AntibodyEnumManager()
for i in range(1, pose.size()+1):
    region = ab_info.get_region_of_residue(pose, i)
    if (region == antibody.cdr_region):
        print(i, enum_manager.cdr_name_enum_to_string(ab_info.get_CDRNameEnum_of_residue(pose, i)))
    else:
        print(i, enum_manager.antibody_region_enum_to_string(region))

print(ab_info.get_CDR_length(antibody.l1))
print(ab_info.get_CDR_cluster(antibody.l1).cluster())


# get CDR sequence
ab_seq = ab_info.get_antibody_sequence()
print(ab_seq)

L1_seq = ab_info.get_CDR_sequence_with_stem(antibody.l1, pose)
print("L1", L1_seq)

for i in range(1, 7):
    cdr = antibody.CDRNameEnum(i)
    print(cdr, ab_info.get_CDR_sequence_with_stem(cdr, pose))


# 你可以采用NeighborhoodResidueSelector来获得邻居的residue
# 也可以通过以下方式获得距离paratope 8A距离以内的epitope
epi_residues = antibody.select_epitope_residues(ab_info, pose, 8)
total=0
for i in range(1, len(epi_residues)+1):
    if epi_residues[i]:
        print(i)
        total+=1
print("Total Epitope Residues:", total)


# 然后针对这些residue计算一些指标
import rosetta.core.simple_metrics.metrics as sm
epi_res_selector = selections.ReturnResidueSubsetSelector(epi_residues)
sasa_metric = sm.SasaMetric(epi_res_selector)
print("\nSASA", sasa_metric.calculate(pose))

total_metric = sm.TotalEnergyMetric(epi_res_selector)
print("\nTOTAL RESIDUE ENERGY", total_metric.calculate(pose))

#Lets use a useful metric to select these residues in pymol
pymol_metric = sm.SelectedResiduesPyMOLMetric(epi_res_selector)
print("\nSELECTION", pymol_metric.calculate(pose))



#此外可以针对每个residue计算SASA
import rosetta.core.simple_metrics.per_residue_metrics as residue_sm
import operator

res_sasa_metric = residue_sm.PerResidueSasaMetric()
res_sasa_metric.set_residue_selector(epi_res_selector)
per_res_sasa = res_sasa_metric.calculate(pose)
#print(per_res_sasa)

#Convert the Map to a Dictionary, which are essentially the same thing. 
for ele in sorted(per_res_sasa.items(), key=operator.itemgetter(1), reverse=False):
    print(ele)






# 同理针对每个residue计算energy
res_energy_metric = residue_sm.PerResidueEnergyMetric()
res_energy_metric.set_residue_selector(epi_res_selector)

per_res_energy = res_sasa_metric.calculate(pose)
#print(per_res_sasa)

#Convert the Map to a Dictionary, which are essentially the same thing. 
for ele in sorted(per_res_energy.items(), key=operator.itemgetter(1), reverse=False):
    print(ele[0], pose.pdb_info().pose2pdb(ele[0]), ele[1])





# CDRResidueSelector
from rosetta.protocols.antibody.residue_selector import *
cdr_selector = CDRResidueSelector(ab_info)
cdr_selector.set_cdrs(h3_l3)
sele = cdr_selector.apply(pose)
for i in range(1, len(sele)):
    if sele[i]:
        print(i, pose.pdb_info().pose2pdb(i))



# AntibodyRegionSelector
region_selector = AntibodyRegionSelector(ab_info)
region_selector.set_region(antibody.antigen_region)
sele = region_selector.apply(pose)

for i in range(1, len(sele)):
    if sele[i]:
        print(i, pose.pdb_info().pose2pdb(i))








# 2. Skeleton RosettaScript Format

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/RosettaScripts

```xml
<ROSETTASCRIPTS>
    <SCOREFXNS>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <SIMPLE_METRICS>
    </SIMPLE_METRICS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
    </MOVERS>
    <PROTOCOLS>
    </PROTOCOLS>
    <OUTPUT />
</ROSETTASCRIPTS>
```

In [ ]:
# 可以采用RosettaScriptsParser类中的generate_mover函数，从xml文件中提取mover指令，并apply到pose
parser = RosettaScriptsParser()
protocol = parser.generate_mover("inputs/min_L1.xml")

if not os.getenv("DEBUG"):
    protocol.apply(pose)

In [ ]:
# 也可以用string的形式直接写一个script
min_L1 = """
<ROSETTASCRIPTS>
	<SCOREFXNS>
	</SCOREFXNS>
	<RESIDUE_SELECTORS>
		<CDR name="L1" cdrs="L1"/>
	</RESIDUE_SELECTORS>
	<MOVE_MAP_FACTORIES>
		<MoveMapFactory name="movemap_L1" bb="0" chi="0">
			<Backbone residue_selector="L1" />
			<Chi residue_selector="L1" />
		</MoveMapFactory>
	</MOVE_MAP_FACTORIES>
	<SIMPLE_METRICS>
		<TimingProfileMetric name="timing" />
		<SelectedResiduesMetric name="rosetta_sele" residue_selector="L1" rosetta_numbering="1"/>
		<SelectedResiduesPyMOLMetric name="pymol_selection" residue_selector="L1" />
		<SequenceMetric name="sequence" residue_selector="L1" />
		<SecondaryStructureMetric name="ss" residue_selector="L1" />
	</SIMPLE_METRICS>
	<MOVERS>
		<MinMover name="min_mover" movemap_factory="movemap_L1" tolerance=".1" /> 
		<RunSimpleMetrics name="run_metrics1" metrics="pymol_selection,sequence,ss,rosetta_sele" prefix="m1_" />
		<RunSimpleMetrics name="run_metrics2" metrics="timing,ss" prefix="m2_" />
	</MOVERS>
	<PROTOCOLS>
		<Add mover_name="run_metrics1"/>
		<Add mover_name="min_mover" />
		<Add mover_name="run_metrics2" />
	</PROTOCOLS>
</ROSETTASCRIPTS>
"""


xml = XmlObjects.create_from_string(min_L1)
protocol = xml.get_mover("ParsedProtocol")

if not os.getenv("DEBUG"):
    protocol.apply(pose)

# 2. Score Function

In [ ]:
# 计算energy function
ras = pose_from_pdb("./inputs/6Q21_A.pdb")
sfxn = get_score_function(True) # Specifying `True` will return the default `ref2015` all-atom energy function, while `False` will specify the default centroid score function.

#当然你也可以自己定义每个原子的weight，但一般没有必要
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)
print(sfxn(ras)) # print total energy of ras
print(sfxn2(ras)) # print the attractive and repulsive energy of ras

sfxn.show(ras) # 计算score是由哪几个部分组成的
print(ras.energies().show(24)) # 单独查看每个residue的energy计算结果

# 另外一种能量计算方式
scorefxn = get_fa_scorefxn()
scorefxn(pose)
energies = pose.energies()
print(energies.residue_total_energies(49))
print(energies.residue_total_energies(49)[pyrosetta.rosetta.core.scoring.fa_dun])



# Note: The _backbone_ hydrogen-bonding terms for each residue are not available from the `Energies` object. 
# You can get them by using EnergyMethodOptions. See http://www.pyrosetta.org/documentation#TOC-Hydrogen-Bonds-and-Hydrogen-Bond-Scoring.

# 获得ATOM之间的作用力, return values of the lj_atr, lj_rep, fa_solv, and fa_elec potentials
etable_atom_pair_energies(ras.residue(1), 1, ras.residue(2), 1, sfxn) 


# Accumulates the unweighted context independent two body interaction energies of <pose> between Residue <rsd1> and Residue <rsd2> into EnergyMap <emap>
emap = EMapVector()
sfxn.eval_ci_2b(pose.residue(res102), pose.residue(res408), pose, emap)
print(emap[fa_atr])
print(emap[fa_rep])
print(emap[fa_sol])

# InterfaceAnalyzerMover 用于计算interface energy
from rosetta.protocols.analysis import InterfaceAnalyzerMover

if not os.getenv("DEBUG"):
    iam = InterfaceAnalyzerMover("LH_ABCDEFGIJKZ")
    iam.set_pack_separated(True)
    iam.apply(pose)
    iam.apply(original_pose)

    dg_term = "dG_separated"
    print("dG Diff:", pose.scores[dg_term] - original_pose.scores[dg_term])
pose.scores # 还可以打印出很多score信息

# 3.Pymol操作

In [ ]:
pymol inputs/rabd/my_ab.pdb inputs/rabd/tutA2_* 
@color_cdrs.pml （or /home/fxu/antibody_design/PyRosetta.notebooks/student-notebooks/inputs/rabd/color_cdrs.pml）
center full_epitope

In [ ]:
pmm = PyMOLMover()
pmm.keep_history(True)
pmm.apply(polyA)

# 4. Pyrosetta一些高级用法

In [ ]:
observer = pyrosetta.rosetta.protocols.moves.AddPyMOLObserver(test2, True) # 当test2这个pose发生改变的时候，可以从pymol中观察到

In [ ]:
# SwitchResidueTypeSetMover，把一个pose转变成low resolution
switch = SwitchResidueTypeSetMover("centroid")
switch.apply(pose)
print(pose.residue(5))

In [ ]:
# small moves, which perturb φ or ψ of a random residue by a random small angle, 
# shear moves, which perturb φ of a random residue by a small angle and ψ of the same residue by the same small angle of opposite sign

kT = 1.0
n_moves = 1
movemap = MoveMap()
movemap.set_bb(True)
small_mover = SmallMover(movemap, kT, n_moves)
shear_mover = ShearMover(movemap, kT, n_moves)

small_mover.angle_max("H", 25)
small_mover.angle_max("E", 25)
small_mover.angle_max("L", 25)
small_mover.apply(test)

################################################
# 也可以设置那些residue可以改变，而那些不能改变
movemap.set_bb(False)
movemap.set_bb(50, True)
movemap.set_bb(51, True)

In [ ]:
# MinMover(), carries out a gradient-based minimization to find the nearest local minimum in the energy function
mm4060 = MoveMap()
mm4060.set_bb_true_range(40, 60)
scorefxn = get_score_function(True) #get the full-atom score function
min_mover.movemap(mm4060)
min_mover.score_function(scorefxn)
print(min_mover)
min_mover.apply(test2)


In [ ]:
# Monte Carlo search, it can decide whether to accept or reject a trial conformation, 
# and it keeps track of the lowest-energy conformation and other statistics about the search
mc = MonteCarlo(test, scorefxn, kT)
mc.boltzmann(test)

In [ ]:
# TrialMover, combines a specified `Mover` with a `MonteCarlo` object
# You can create a `TrialMover` from any other type of `Mover`
trial_mover = TrialMover(small_mover, mc)
for i in range(10):
    trial_mover.apply(test)
    
print(trial_mover.num_accepts())
print(trial_mover.acceptance_rate())

In [ ]:
# Sequence Movers, 我的理解是把pyrosetta中的各种mover串起来
seq_mover = SequenceMover()
seq_mover.add_mover(small_mover)
seq_mover.add_mover(shear_mover)
seq_mover.add_mover(min_mover)
print(seq_mover)

# Repeat Movers 
n_repeats = 3
repeat_mover = RepeatMover(trial_mover, n_repeats)
print(repeat_mover)

In [ ]:
# relax 似乎是对pose进行结构上的refinement
# classification relax比较费时，可以考虑采用FastRelax()
sfxn = get_fa_scorefxn()
pose = pose_from_pdb("inputs/1YY8.clean.pdb")
relax = pyrosetta.rosetta.protocols.relax.ClassicRelax()
relax.set_scorefxn(sfxn)
relax.apply(pose)



#############################################################
# FastRelax()
scorefxn = get_score_function()
pose = original_pose.clone()
fr = FastRelax()
fr.set_scorefxn(scorefxn)
fr.max_iter(100)
fr.apply(pose)


############################################################
# 手动做regional relax，可能会产生过度优化
fr = FastRelax()
fr.set_scorefxn(scorefxn)
fr.max_iter(100)
cdr_selector = CDRResidueSelector()
cdr_selector.set_cdr(h1)
mmf = MoveMapFactory()
mmf.add_bb_action(mm_enable, cdr_selector) # mm_enable and mm_disable are Enums (numbered variables) that come when we import the MMF
mmf.add_chi_action(mm_enable, cdr_selector) # We are taking this out for speed.
mm  = mmf.create_movemap_from_pose(pose)
fr.set_movemap_factory(mmf)
fr.set_task_factory(pack_cdrs_and_neighbors_tf)
fr.apply(pose)





############################################################
# 因此可以采用cartesian-space
pose = original_pose.clone()
cart_sf = create_score_function("ref2015_cart")
mmf.set_cartesian(True)
fr.set_movemap_factory(mmf)
fr.set_scorefxn(cart_sf)
fr.cartesian(True)

#This is a general recommendation for cartesian minimization - it lowers the number of maximum cycles.
# More than this only increases time of protocol, but has little effect on energies/structure
fr.max_iter(200)
fr.apply(pose)




#############################################################
# 也可以采用classic relax，用于antibody，但是看不懂。。。。。
pose = original_pose.clone()
ab_info = AntibodyInfo(pose)
ft = FoldTree()
start = ab_info.get_CDR_start(h1, pose)
stop =  ab_info.get_CDR_end(h1, pose)
cutpoint = int((stop-start)/2) + start
cdr_loop = Loop(start, stop, cutpoint)
cdr_loops = Loops()
cdr_loops.add_loop(cdr_loop)

fold_tree_from_loops(pose, cdr_loops, ft)
pose.fold_tree(ft)
add_cutpoint_variants(pose)

#Add chainbreak term so we don't get wacky stuff.  This term helps keep the peptide closed during bb movement.
scorefxn_ch = scorefxn
scorefxn_ch.set_weight(rosetta.core.scoring.chainbreak, 100)

#Setup our FastRelax again for dihedral-space.
mmf.set_cartesian(False)
fr.set_scorefxn(scorefxn_ch)
fr.cartesian(False)
fr.set_movemap_factory(mmf)
fr.max_iter(0) #Reset to default - if its 0, then we don't set it in the MinMover that FastRelax runs

#Skip for tests
if not os.getenv("DEBUG"):
    fr.apply(pose)
original_ft = pose.fold_tree()
pose.fold_tree(original_ft)
pose.dump_pdb('outputs/2r0l_dih_rel_H1.pdb')

In [ ]:
# packing: 是对侧链的结构优化，并不会改变sequence构成
tf = TaskFactory() # 用来定义具体对pose中哪些残基进行packing
tf.push_back(operation.InitializeFromCommandline()) # 首先读取init中的一些参数
tf.push_back(operation.RestrictToRepacking()) # 只做packing，不做design

#初始化一个packer，并应用到pose
packer = pack_min.PackRotamersMover()
packer.task_factory(tf)
if not os.getenv("DEBUG"):
    packer.apply(pose)
pose.dump_pdb('/josiefanxu_outputs/2r0l_all_repack.pdb')


#######################################################
# 当然可以来约束对指定的residue进行packing-->regional packing
# 例如选择CDR区及其neighbourhood区域
cdr_selector = CDRResidueSelector()
cdr_selector.set_cdr(h1)
nbr_selector = selections.NeighborhoodResidueSelector()
nbr_selector.set_focus_selector(cdr_selector)
nbr_selector.set_include_focus_in_subset(True)
prevent_repacking_rlt = operation.PreventRepackingRLT()
prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, nbr_selector, True )
tf.push_back(prevent_subset_repacking)
pack_cdrs_and_neighbors_tf = tf.clone()
packer.task_factory(tf)
if not os.getenv("DEBUG"):
    packer.apply(pose)
pose.dump_pdb('/josiefanxu_outputs/2r0l_CDR_repack.pdb')

########################################################
# 可以在packing中加入design
tf.clear()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(prevent_subset_repacking)

#Turn off design of neighbors
nbr_selector2 = selections.NeighborhoodResidueSelector()
nbr_selector2.set_focus_selector(cdr_selector)
nbr_selector2.set_include_focus_in_subset(False)
restrict_to_repack = operation.RestrictToRepackingRLT()
prevent_nbr_design = operation.OperateOnResidueSubset(restrict_to_repack, nbr_selector2, False )
tf.push_back(prevent_nbr_design)
packer_new = pack_min.PackRotamersMover()
packer_new.task_factory(tf)
if not os.getenv("DEBUG"):
    packer_new.apply(pose)
pose.dump_pdb('/josiefanxu_outputs/2r0l_CDR_repack_design.pdb')



########################################################
# 小tips：提取CDR区的residue
cdr_res = []
print("CDR")
for i in select.get_residue_set_from_subset(cdr_selector.apply(pose)):
    print(i)
    cdr_res.append(i)
    
print("\nCDR+Neighbors")
for i in select.get_residue_set_from_subset(nbr_selector.apply(pose)):
    if i in cdr_res:
        print(i,"CDR")
    else:
        print(i)

In [ ]:
# protein design

!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
import logging
logging.basicConfig(level=logging.INFO)
import pyrosetta
import pyrosetta.toolbox
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0")

# 读取pdb文件
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

# 我的理解是利用resfile来指定需要进行design的residue ID
resfile = "./josiefanxu_outputs/resfile"
with open(resfile, "w") as f:
    f.write("NATAA\n")
    f.write("start\n")
    for i in cys_res:
        f.write("{0} {1} ALLAAxc\n".format(i, pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)[1]))
!cat {resfile}

# 设置TaskFactory，
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.ReadResfile(resfile))
packer_task = tf.create_task_and_apply_taskoperations(pose)

# 设置MoveMapFactory，
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True)
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

# 设置FastRelax
fr = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1)
fr.cartesian(True)
fr.set_task_factory(tf)
fr.set_movemap_factory(mmf)
fr.min_type("lbfgs_armijo_nonmonotone") # For non-Cartesian scorefunctions, use "dfpmin_armijo_nonmonotone"
fr.apply(pose)



########################################################
# 更多的一些设置
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())


# Disable design (i.e. repack only) on not_chain_A_cys_res
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), not_chain_A_cys_res))

# Enable design on chain_A_cys_res
aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
aa_to_design.aas_to_keep("ADEFGHIKLMNPQRSTVWY")
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    aa_to_design, chain_A_cys_res))

# Convert the task factory into a PackerTask
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)

rel_design = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1, script_file="MonomerDesign2019")
rel_design.cartesian(True)
rel_design.set_task_factory(tf)
rel_design.set_movemap(mm)
rel_design.minimize_bond_angles(True)
rel_design.minimize_bond_lengths(True)



########################################################
# 小tips：提取chain和residue：get_residues_from_subset
disulfide_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS:disulfide")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)))

cys_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(cys_res.apply(pose)))

chain_A = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A.apply(pose)))
chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(selector1=cys_res, selector2=chain_A)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A_cys_res.apply(pose)))

not_chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(chain_A_cys_res)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(not_chain_A_cys_res.apply(pose)))

# disclose二硫键
for i in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
    for j in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
        if pyrosetta.rosetta.core.conformation.is_disulfide_bond(pose.conformation(), i, j):
            pyrosetta.rosetta.core.conformation.break_disulfide(pose.conformation(), i, j)
